In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

In [3]:
!git clone https://github.com/Anil-Banjade/Nepali_Multi_Modal.git


Cloning into 'Nepali_Multi_Modal'...
remote: Enumerating objects: 585, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 585 (delta 7), reused 6 (delta 6), pack-reused 574 (from 1)
Receiving objects: 100% (585/585), 3.86 MiB | 10.99 MiB/s, done.
Resolving deltas: 100% (366/366), done.


In [4]:
!git fetch origin
!git reset --hard origin/main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [5]:
%cd Nepali_Multi_Modal

/content/Nepali_Multi_Modal


In [6]:
%%capture
!pip install -r requirements.txt

In [7]:
from src.multimodal_text_generation.main import main

Using device: cpu


In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import dataset,DataLoader

from src.multimodal_text_generation.config import config
from src.multimodal_text_generation.models.transformer import Transformer
from src.multimodal_text_generation.data.dataset import CaptionEmbeddingDataset, collate_fn
from src.multimodal_text_generation.utils.inference import run_inference
from src.multimodal_text_generation.trainer import train_model

In [ ]:
def main(resume_checkpoint=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tokenizer = AutoTokenizer.from_pretrained('NepBERTa/NepBERTa')
    text_model = AutoModel.from_pretrained("NepBERTa/NepBERTa", from_tf=True)

    loaded_results = torch.load('/content/drive/MyDrive/Minor_project/aligned_caption_embeddings.pt')
    
    model = Transformer(tokenizer)
    if resume_checkpoint:
        model.load_state_dict(torch.load(resume_checkpoint))

    dataset = CaptionEmbeddingDataset(loaded_results,tokenizer)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

    model = Transformer(tokenizer)

    num_epochs = 20
    train_model(model, dataloader, num_epochs, device)
    return model

In [ ]:
if __name__ == "__main__": 
    model = main('/content/drive/MyDrive/Minor_project/autoregressive_gen_e_model.pt')
    torch.save(model.state_dict(), '/content/drive/MyDrive/Minor_project/autoregressive_gen_e_model_v2.pt')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/534M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
<ipython-input-9-5c0a2b0e50b0>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend yo

Epoch [0/20], Batch [0/10111], Loss: 10.7592
Epoch [0/20], Batch [1000/10111], Loss: 2.6533
Epoch [0/20], Batch [2000/10111], Loss: 2.4107
Epoch [0/20], Batch [3000/10111], Loss: 2.4759
Epoch [0/20], Batch [4000/10111], Loss: 1.9427
Epoch [0/20], Batch [5000/10111], Loss: 2.5095
Epoch [0/20], Batch [6000/10111], Loss: 2.0181
Epoch [0/20], Batch [7000/10111], Loss: 1.3500
Epoch [0/20], Batch [8000/10111], Loss: 1.1217
Epoch [0/20], Batch [9000/10111], Loss: 2.1635
Epoch [0/20], Batch [10000/10111], Loss: 1.5676
Epoch [0/20], Average Loss: 2.1873
Epoch [1/20], Batch [0/10111], Loss: 1.6331
Epoch [1/20], Batch [1000/10111], Loss: 1.7759
Epoch [1/20], Batch [2000/10111], Loss: 1.9993
Epoch [1/20], Batch [3000/10111], Loss: 1.9917
Epoch [1/20], Batch [4000/10111], Loss: 1.7874
Epoch [1/20], Batch [5000/10111], Loss: 1.1986
Epoch [1/20], Batch [6000/10111], Loss: 1.3518
Epoch [1/20], Batch [7000/10111], Loss: 1.5941
Epoch [1/20], Batch [8000/10111], Loss: 1.4824
Epoch [1/20], Batch [9000/101

In [9]:
import numpy as np

In [21]:
def test():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    loaded_results = torch.load(
        '/content/drive/MyDrive/Minor_project/aligned_caption_embeddings.pt',
        weights_only=False
    )

    tokenizer = AutoTokenizer.from_pretrained('NepBERTa/NepBERTa')
    dataset = CaptionEmbeddingDataset(loaded_results,tokenizer)
    for i in range(1000,1010):
      test_caption, test_embedding = dataset[i]


      if isinstance(test_embedding, np.ndarray):
          test_embedding = torch.from_numpy(test_embedding)

      fused_embedding = test_embedding.clone().detach().to(device)

      model_path = '/content/drive/MyDrive/Minor_project/autoregressive_gen_e_model.pt'
      generated_caption = run_inference(model_path, fused_embedding, device)

      print("Original:", test_caption)
      print("Generated:", generated_caption)

In [22]:
test()

Loading model...

Attempt 1/3
Original: एउटा कालो र खैरो कुकुर एउटा सेतो गालामा बिछ्याइएको छ।
Generated: एउटा मानिस चटटान चढको परखालमा चढद छ भन एउटा ठला कटा भइमा उभिरहको छ
Loading model...

Attempt 1/3
Original: एउटा खैरो र कालो कुकुर ढाडको गालामा सुतिरहेको छ।
Generated: एउटा ठलो ककर एउटा सानो ककरलाई हरद
Loading model...

Attempt 1/3
Original: एउटा सानो खैरो र कालो कुकुर रङमा पल्टिएको।
Generated: एउटा सानो कटाल पारकमा फटबल बललाई लात हानको छ ।
Loading model...

Attempt 1/3
Original: एउटा सानो कुकुर सेतो कार्पेटमा बसिरहेको छ।
Generated: एउटा मानिस र उसको ककर बनचबाट सरयासत हरद ।
Loading model...

Attempt 1/3
Original: एउटा सानो कुकुरले सेतो शेग कार्पेटमा टाउको हल्लाउँछ।
Generated: एउटा ककर बालवामा दौडनछ ।
Loading model...

Attempt 1/3
Original: नुहाउने सूटमा मुस्कुराउँदै केटीहरू बालुवामा बसिरहेका
Generated: एउटा कालो ककर हिउल भरिएको पहाडमा हाम फालछ ।
Loading model...

Attempt 1/3
Original: साथीहरूको समूह समुद्र तटमा तस्विरको लागि मुस्कुराउँदै।
Generated: एउटा ककर हिउमा खलौना समाउन उफरनछ ।